# Spatial Data Management: Term Project (Fall 2022)

**Lukman Fashina**

### Earth Engine Application in Studying Precipitation Pattern and Agricultural Practice in the U.S.



#### Problem Statement

More than 75% of cropped land globally is under rain-fed agriculture (Reddy & Syme, 2015), and variability in precipitation volume under this practice can affect grain/crop yield. This (variability) is more likely to be the norm with climate change. However, an age-long strategy to combat water scarcity for agrciultural practice is irrigation. Although this is a positive climate adaptation strategy, there is an inherent negative impact on both ground- and surface water resources if not well managed.

The United States' 166 million hectares of net cropland area ranks second in the world behind India (US Geological Survey [USGS], n.d). Furthermore, the US is the world’s largest producer of corn and second-largest producer of soybeans (U.S. Department of Agriculture [USDA], n.d) which suggests the intensive nature of agriculture in the country. So, is all the cropland area in the U.S. rain-fed or likely irrigated? To answer this question, more questions are asked which are as follows:

##### Research Questions
Which state has the highest amount of cropland in the U.S.?

What is the precipitation pattern across the U.S. states?

What type of farming practices are likely taking place in cropland areas of U.S. states based on rainfall pattern?

#### Datasets:

CHIRPS Pentad: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final)
https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD#description


PRISM Monthly Spatial Climate Dataset AN81m:
https://developers.google.com/earth-engine/datasets/catalog/OREGONSTATE_PRISM_AN81m#bands

USDA National Agricultural Statistics Service Cropland Data Layer. {YEAR}. Published crop-specific data layer [Online]. Available at https://nassgeodata.gmu.edu/CropScape/ (accessed {DATE}; verified {DATE}). USDA-NASS, Washington, DC.
The Cropland Data Layer (CDL) is an annually updated GIS dataset by the USDA that maps out crops grown in the United States

TIGER: US Census States 2018: https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States

##### Data Analysis

Zonal Statistics: This is used to calculated the sum statistics on cell values of cropland data layer(a raster) within the zones defined by US Census States 2018.

In [1]:
# To run analysis in Colab/Jup Notebook environment:
# Then Install/import geemap package by uncommenting the folllowing to use geemap and all the necessary dependencies:


# import subprocess

# try:
#     import geemap
# except ImportError:
#     print('geemap package not installed. Installing ...')
#     subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# %pip install pygis

In [1]:
# Import libraries and initialize Earth Engine Python API:
import ee
import geemap
import pandas as pd
import geemap.foliumap as geemap
geemap.ee_initialize()

###### Visualising U.S. PRISM (Climate) Precipitation Data: Map 1

In [2]:
# Create an interactive map
Map = geemap.Map()

# Import the US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')
Map.add_labels(
    data=states,
    column="STUSPS",
    font_size="10pt",
    font_color="blue",
    font_family="arial",
    font_weight="bold",
    draggable=True,
)
# Import the PRISM daily and monthly climate datasets, set visualization and region of interest parameters:

dataset = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m')
precipitation = dataset.select('ppt').first().clipToCollection(states)

precipVis = {
  'min': 0.0,
  'max': 300.0,
  'palette': ['red', 'yellow', 'green', 'cyan', 'purple'],
}
Map.setCenter(-100.55, 40.71, 4)
Map.addLayer(precipitation, precipVis, 'Precipitation')
Map.add_colorbar(precipVis, label="precipitation (mm)", layer_name="PRISM Long-Term Precip")
Map

##### "Zonal Statistics by Group" approach to Compute the Cropland Cover Composition of US states

The input image is the USDA NASS Cropland Data Layers.

The input zone layer is the US states.




Figure 1: Cropland Area of U.S. States

Figure 2: A Bar Chart of States by Cropland Area

In [3]:
Map = geemap.Map(center=[40, -100], zoom=3)

# Import the USDA NASS Cropland Data
usdaLandcover = ee.ImageCollection('USDA/NASS/CDL') \
                  .filter(ee.Filter.date('2018-01-01', '2020-01-01')) \
                  .first() \
                  .select('cropland')

Map.addLayer(usdaLandcover, {}, 'Crop Landcover')

# Import the US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')

# Add Cropland legend
Map.add_legend(title='USDA Cropland', builtin_legend='USDA/NASS/CDL')
Map

In [4]:
# statistics_type computation = Sum (the total area of each cropland in each state) 

usdaLC_stats = 'usdacrplnd.csv'

geemap.zonal_stats_by_group(
    usdaLandcover,
    states,
    usdaLC_stats,
    statistics_type='SUM',
    denominator=1e6,
    decimal_places=2,
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to c:\Users\lukma\Desktop\GEE Projects\usdacrplnd.csv


###### Figure 2: A Bar Chart of States by Cropland Area

In [5]:
geemap.bar_chart('usdacrplnd.csv', x='NAME', y='Class_sum', max_rows=30, x_label='States', y_label='Cropland Area (km2)')

###### Figure 3: Cropland Area and Precipitation Pattern of U.S. States

Data: PRISM Monthly Spatial Climate Dataset AN81m


USDA National Agricultural Statistics Service Cropland Data Layer

In [6]:
Map = geemap.Map()
Map.split_map(left_layer='HYBRID', right_layer='TERRAIN')

In [7]:
Map = geemap.Map(center=(40, -100), zoom=3, height=600)
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')
Map.add_labels(
    data=states,
    column="STUSPS",
    font_size="9pt",
    font_color="blue",
    font_family="arial",
    font_weight="bold",
    draggable=True,
)

dataset = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m') 

precipdat = dataset.select('ppt').first().clipToCollection(states)

usdaLandcover = ee.ImageCollection('USDA/NASS/CDL') \
                  .filter(ee.Filter.date('2020-01-01', '2022-01-01')) \
                  .first() \
                  .select('cropland')

Crop_clip = usdaLandcover.clipToCollection(states)


precipitationVis = {
  'min': 0.0,
  'max': 300.0,
  'palette': ['red', 'yellow', 'green', 'cyan', 'purple'],
}

usdaLandcover_viz = {'bands': ['cropland']}
left_layer = geemap.ee_tile_layer(precipdat, precipitationVis, 'Precipitation')
right_layer = geemap.ee_tile_layer(Crop_clip, usdaLandcover_viz, 'Cropland')

Map.split_map(left_layer, right_layer)
Map.add_legend(title='USDA Cropland', builtin_legend='USDA/NASS/CDL', position= 'bottomleft')
Map.add_colorbar(precipitationVis, label="precipitation (mm)", layer_name="PRISM Long-Term Precip")
Map

### Results

##### Question 1) Which state has the highest amount of cropland in the U.S.?

Among the top 30 states in terms of cropland area size, Texas ranks first as shown on the Figure 2, followed by California, and Montana among others.  


##### Question 2) What is the precipitation pattern across the U.S. states?

As observable from Figure 1, much of the states in The Great Plains receive rainfall of less than 75mm, while those in the south east U.S. receive rainfall slightly above 150mm, and a few in the Pacific Northwest receive close to 300mm of rainfall.

##### Question 3) what type of farming practice are likely taking place in cropland areas of U.S. states based on rainfall pattern?

It is highly likely that much of the states with the largest cropland area (mostly in The Great Plains and western U.S.) are under irrigation farming since they receive the smallest amount rainfall (jointly shown in Figure 3) which may be insufficent for large scale year-round cropping enterprises. If this is the case, it is imperative that sustainability is factored into the existing water rights and allocation legal framework to discourage over-abstraction of groundwater and surface water reources. 

### References

CHIRPS Pentad: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final)
https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD#description



PRISM Monthly Spatial Climate Dataset AN81m:
https://developers.google.com/earth-engine/datasets/catalog/OREGONSTATE_PRISM_AN81m#bands


USDA National Agricultural Statistics Service Cropland Data Layer. {YEAR}. Published crop-specific data layer [Online]. Available at https://nassgeodata.gmu.edu/CropScape/ (accessed {DATE}; verified {DATE}). USDA-NASS, Washington, DC.


USGS (n.d): Map of Croplands in the United States https://www.usgs.gov/media/images/map-croplands-united-states

USDA (n.d). Corn and soybean production up in 2021. https://www.nass.usda.gov/Newsroom/2022/01-12-2022.php https://www.ers.usda.gov/topics/crops/soybeans-and-oil-crops/

Reddy, V.R, and Syme, G.J. (2015).Integrated Assessment of Scale Impacts of Watershed Intervention
https://www.sciencedirect.com/book/9780128000670/integrated-assessment-of-scale-impacts-of-watershed-intervention